In [1]:
#Load the librarys
import pandas as pd #To work with dataset
import numpy as np #Math library
import matplotlib.gridspec as gridspec
import seaborn as sns #Graph library that use matplot in background
import matplotlib.pyplot as plt #to plot some parameters in seaborn
import warnings
# Preparation  
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer, StandardScaler,Normalizer,RobustScaler,MaxAbsScaler,MinMaxScaler,QuantileTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import KBinsDiscretizer
# Import StandardScaler from scikit-learn

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer,IterativeImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import make_column_transformer,ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline,FeatureUnion
from sklearn.manifold import TSNE
# Import train_test_split()
# Metrics
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import roc_curve,confusion_matrix
from datetime import datetime, date
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.linear_model import LogisticRegression

#import tensorflow as tf 
#from tensorflow.keras import layers
#from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow.keras.callbacks import LearningRateScheduler
#import smogn
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
# For training random forest model
import lightgbm as lgb
from scipy import sparse
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans 
# Model selection
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression,f_classif,chi2
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import mutual_info_classif,VarianceThreshold

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
import lightgbm as lgbm
#from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from xgboost import XGBClassifier,XGBRegressor
from sklearn import set_config
from itertools import combinations
# Cluster :
from sklearn.cluster import MiniBatchKMeans
#from yellowbrick.cluster import KElbowVisualizer
#import smong 
import category_encoders as ce
import warnings
import optuna 
from joblib import Parallel, delayed
import joblib 
from sklearn import set_config
from typing import List, Optional, Union
set_config(display='diagram')
warnings.filterwarnings('ignore')


##  Load the data

In [2]:
%%time 
train = pd.read_csv('../input/GiveMeSomeCredit/cs-training.csv')
test = pd.read_csv('../input/GiveMeSomeCredit/cs-test.csv')
train.head(3)

CPU times: user 267 ms, sys: 69.4 ms, total: 337 ms
Wall time: 515 ms


,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0


##  Convert Dtypes :

In [3]:
# Convert Dtypes :
train[train.select_dtypes(['int64','int16','float16','float32','float64','int8']).columns] = train[train.select_dtypes(['int64','int16','float16','float32','float64','int8']).columns].apply(pd.to_numeric)
train[train.select_dtypes(['object','category']).columns] = train.select_dtypes(['object','category']).apply(lambda x: x.astype('category'))
# Convert Dtypes :
test[test.select_dtypes(['int64','int16','float16','float32','float64','int8']).columns] = test[test.select_dtypes(['int64','int16','float16','float32','float64','int8']).columns].apply(pd.to_numeric)
test[test.select_dtypes(['object','category']).columns] = test.select_dtypes(['object','category']).apply(lambda x: x.astype('category'))

## Reduce memory

In [4]:
# Author : https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        name =df[col].dtype.name 
        
        if col_type != object and col_type.name != 'category':
        #if name != "category":    
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

train= reduce_mem_usage(train)
test= reduce_mem_usage(test)

Memory usage of dataframe is 13.73 MB
Memory usage after optimization is: 3.29 MB
Decreased by 76.0%
Memory usage of dataframe is 9.29 MB
Memory usage after optimization is: 2.90 MB
Decreased by 68.7%


# let's predict monthly income: 


In [5]:
%%time 
train=train.drop(['Unnamed: 0','SeriousDlqin2yrs'], axis=1)
test=test.drop(['Unnamed: 0','SeriousDlqin2yrs'], axis=1)

CPU times: user 7.71 ms, sys: 103 µs, total: 7.81 ms
Wall time: 7.15 ms


In [6]:
train_final= pd.concat( [train, test], axis=0) 
testdf_income= train_final[train_final['MonthlyIncome'].isnull()==True]
traindf_income = train_final[train_final['MonthlyIncome'].isnull()==False]
y_income = traindf_income['MonthlyIncome']
X=traindf_income.drop(["MonthlyIncome"],axis=1)
X_test=testdf_income.drop(["MonthlyIncome"],axis=1)

In [7]:
# select non-numeric columns
cat_columns = X.select_dtypes(exclude=['int64','int16','float16','float32','float64','int8']).columns
cat_columns

Index([], dtype='object')

**Num Features**



In [8]:
# select the float columns
num_columns = X.select_dtypes(include=['int64','int16','float16','float32','float64','int8']).columns
num_columns

Index(['RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents'],
      dtype='object')

##  Define preprocess

In [9]:
#Define vnum pipeline
numeric_transformer1 = Pipeline(
                            steps=[
                            ('imputer', SimpleImputer(strategy='median'
                                                      ,add_indicator=True)),
                            ('scaler', PowerTransformer()),#(Numerical Input, Numerical Output)
                            # Create an SelectKBest object to select features with two best ANOVA F-Values
                            #The F-value scores examine if, when we group the numerical feature by the target vector, the means for each group are significantly different
                           # ('reducedim',  SelectPercentile(f_classif,percentile=90))
                            ]
                            )
numeric_transformer2 = Pipeline(
                            steps=[
                            ('imputer', SimpleImputer(strategy='median'
                                                      ,add_indicator=True)),
                            ('scaler', PowerTransformer()),#(Numerical Input, Numerical Output)
                            # Create an SelectKBest object to select features with two best ANOVA F-Values
                            #The F-value scores examine if, when we group the numerical feature by the target vector, the means for each group are significantly different
                           # ('reducedim',  SelectPercentile(f_classif,percentile=90))
                            ]
                            )

In [10]:
numeric_transformer1.fit(X)
X_pre = numeric_transformer1.transform(X)
#X_test_pre = numeric_transformer1.transform(X_test)

## CV Design

In [11]:
cross_validation_design = StratifiedKFold( n_splits=5,
                                           shuffle=True
                                        ,random_state=1)
cross_validation_design_regression=KFold( n_splits=7,
                                           shuffle=True
                                        ,random_state=1)

# Optuna

In [12]:
 def objective_xgbr_income(trial,data=X_pre,target=y_income):
    # Optuna+cv :
    #https://www.kaggle.com/hamidrezabakhtaki/xgboost-catboost-lighgbm-optuna-final-submission
    # https://www.kaggle.com/prashant111/lightgbm-classifier-in-python
    # https://www.kaggle.com/tunguz/tps-09-21-histgradientboosting-with-optuna
    X_train,X_test, y_train,  y_test = train_test_split(data, target, 
                                                        test_size=0.25,
                                                       random_state=42,shuffle=False)
    param = {   "verbosity": 0,
                #"objective": "binary:logistic",
                #"eval_metric": "auc",
                'random_state': 48,
                # regression
                'objective':'reg:squarederror', 
                'eval_metric': 'mae',
                #early_stopping_rounds=100 ,
               # 'gpu_id':0, 
               # 'predictor':"gpu_predictor",
                # use exact for small dataset.
                #"tree_method": "exact",
                # big data :
                #'tree_method': 'gpu_hist',
                # defines booster, gblinear for linear functions.
                "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
                # L2 regularization weight.
                "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
                # L1 regularization weight.
                "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
                # sampling ratio for training data.
                "subsample": trial.suggest_float("subsample", 0.2, 1.0),
                # sampling according to each tree.
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
                "learning_rate" : trial.suggest_float("learning_rate" , 0.005 , 0.2),
                "n_estimators" : trial.suggest_int("n_estimators" , 1000 ,10000),
             }
    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
             
    pipe_xgbr = Pipeline(
                    steps=[
                        ('preprocessor', numeric_transformer1),
                        ('classifier', XGBRegressor(
                      #objective="mae",
                       **param))
                    ]
                )
    
    #mae =cross_val_score(pipe_xgbr, data, target,cv=cross_validation_design_regression,scoring='neg_mean_absolute_error').mean()
    model  = XGBRegressor(
                      #objective="mae",
                       **param)
    
    model.fit( X_train, y_train,
              eval_set=[(X_test, y_test)],
              early_stopping_rounds=50,
              verbose=False)
             
    #pipeline_model_lgbm.fit(X_train,y_train)
    #preds = model.predict_proba(X_test)[:,1]
    y_pred= model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    #oof[test_idx] = preds
    #average_precesion = average_precision_score(y_true= y_test, y_score= preds)
    return mae                

In [13]:
%%time 
study_xgbr_income = optuna.create_study(direction="minimize")
study_xgbr_income.optimize(objective_xgbr_income ,n_trials=2)

[I 2021-12-23 17:10:40,830] A new study created in memory with name: no-name-55bb99fc-66f3-4401-a7ad-d2caae10268f
[I 2021-12-23 17:10:44,421] Trial 0 finished with value: 2766.580810546875 and parameters: {'booster': 'gbtree', 'lambda': 0.00238488501352787, 'alpha': 1.6813977864281414e-05, 'subsample': 0.8326222301309343, 'colsample_bytree': 0.4201503484953481, 'learning_rate': 0.17654040301896085, 'n_estimators': 3255, 'max_depth': 3, 'min_child_weight': 9, 'eta': 4.736422393228843e-08, 'gamma': 0.00012834758878292415, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 2766.580810546875.
[I 2021-12-23 17:10:58,702] Trial 1 finished with value: 3071.292724609375 and parameters: {'booster': 'dart', 'lambda': 0.0005153181960410947, 'alpha': 0.1728839851175947, 'subsample': 0.21889639174409511, 'colsample_bytree': 0.2869690790730113, 'learning_rate': 0.1247548080263241, 'n_estimators': 9089, 'max_depth': 9, 'min_child_weight': 3, 'eta': 2.364682934545983e-08, 'gamma': 0.000189339144

CPU times: user 1min 10s, sys: 96.8 ms, total: 1min 10s
Wall time: 17.9 s


In [14]:
print("Numbers of finished trials : " , len(study_xgbr_income.trials))
print("Best Trials : ", study_xgbr_income.best_trial.params)
print("Best Values : " , study_xgbr_income.best_value)

Numbers of finished trials :  2
Best Trials :  {'booster': 'gbtree', 'lambda': 0.00238488501352787, 'alpha': 1.6813977864281414e-05, 'subsample': 0.8326222301309343, 'colsample_bytree': 0.4201503484953481, 'learning_rate': 0.17654040301896085, 'n_estimators': 3255, 'max_depth': 3, 'min_child_weight': 9, 'eta': 4.736422393228843e-08, 'gamma': 0.00012834758878292415, 'grow_policy': 'depthwise'}
Best Values :  2766.580810546875


In [15]:
#testdf_NumberOfDependents= train_final[train_final['NumberOfDependents'].isnull()==True]
#traindf_NumberOfDependents = train_final[train_final['NumberOfDependents'].isnull()==False]
#y_NumberOfDependents = traindf_NumberOfDependents['NumberOfDependents']
#traindf_NumberOfDependents.drop("NumberOfDependents",axis=1,inplace=True)

In [16]:
# Save the predictions to a CSV file
#sub = pd.read_csv('../input/GiveMeSomeCredit/sampleEntry.csv')
#sub['Probability']=predictions
#sub.to_csv('pipe_lgbm_catboost_qauntile_new.csv', index=False)
#sub